In [ ]:
from utils.tf import *
from utils.db import *
from utils.gd2 import *
from utils.clt_repetidos import *
from datetime import datetime, time, timedelta
import cx_Oracle
import pandas as pd
import os
from utils.clt_repetidos import * 
from datetime import datetime
import polars as pl

In [3]:
df_clientes =  pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/ETL/11_04_2025/CLientesT_FINAL_ACTIVOS.csv")

In [72]:
df_clientes["FECHA_NAC_O_CONST"] = df_clientes["FECHA_NAC_O_CONST"].apply(
    lambda x: (
        x if isinstance(x, str) and "NO SE TIENE REGISTRADA LA INFORMACION" in x
        else pd.to_datetime(str(x).split(",")[0].strip(), errors='coerce').date()
    )
)

In [16]:
df_copy_clientes = df_clientes.copy()

In [56]:
rutas_dfs = {
    "MECA":'/home/ale1726/proyects/datalake/clientes/data/productos/MECA/data/21_04_2025/productos_clientes_meca.dat',
    "SIAG":'/home/ale1726/proyects/datalake/clientes/data/productos/SIAG/data/05_02_2025/productos_clientes_SIAG.dat',
    "SIMS":'/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/data/21_04_2025/productos_clientes_sims.dat',
    "SIPE":'/home/ale1726/proyects/datalake/clientes/data/productos/SIPE/data/22_04_2025/productos_clientes_sipe.dat' ,
    "SIRAC":'/home/ale1726/proyects/datalake/clientes/data/productos/SIRAC/data/05_02_2025/productos_clientes_SIRAC.dat',
    "SOI": '/home/ale1726/proyects/datalake/clientes/data/productos/SOI/data/22_04_2025/productos_clientes_soi.dat',
    "TAS" : '/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat'
    }

name_busqueda =  ".+ TASA ACTIVO, S.A. DE C.V., F.I.R.V."
#name_busqueda = 'GARANTIAS CREDITO'

In [ ]:
df_copy_clientes["SOXID"] = df_copy_clientes.apply(
    lambda row: {f"{so}:{id}" for so, id in zip(row["SISTEMA_ORIGEN"].split(","), row["NUMERO_CLIENTE"].split(","))},
    axis=1
)

In [ ]:
resultado = buscar_cliente(name_busqueda, rutas_dfs, df_copy_clientes)

In [86]:
df =  pd.read_csv(rutas_dfs["TAS"],  low_memory=False)

In [87]:
df

,NUMERO_CLIENTE,ESPROVEED,TIPO_PERSONA,NOMBRE,NOMLARGO,RFC,NLINEA,ICONTRATO,IORDEN,FOPER,...,DIA_FVENCE,IINSTR,ITINSTR,NTINSTR,TASA,MONEDA,MONTO,MONTO_ASIGNADO,MONTO_REAL,FECHA_REGISTRO
0,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,01040153,1040153,430215,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,3.302570e+09,3.302570e+09,2020-11-12
1,51000,False,Cuentas Propias,GARANTIAS CREDITO,GARANTIAS CREDITO,NFI3406305T0,00051000,51000,430214,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.297121e+09,3.297121e+09,3.297121e+09,2020-11-12
2,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,01040153,1040153,430100,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,0.000000e+00,3.308033e+09,2020-11-12
3,51000,False,Cuentas Propias,GARANTIAS CREDITO,GARANTIAS CREDITO,NFI3406305T0,00051000,51000,430063,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,0.000000e+00,3.308033e+09,2020-11-12
4,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,01040153,1040153,329069,2020-04-01,...,25,DEPBANX2,OTROS,Otros Instrumentos,6.50,MXP,5.217592e+09,5.217592e+09,5.217592e+09,2020-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1904349,1053131,True,Moral Nacional no Gravable,FEFA,FONDO ESPECIAL PARA EL FINANCIAMIENTO AGROPECU...,FEF650826K41,01053131,1053131,298161,2006-06-09,...,16,,OTROSBANC,Instrumentos con man fisc,7.03,MXP,1.230000e+09,0.000000e+00,1.230000e+09,2006-06-09
1904350,1062813,True,Moral Nacional no Gravable,BLK / BLKLIQP,"BLK FONDEO GUBERNAMENTALES MEXICO EXENTOS, S.A...",HLP010918LTA,01062813,1062813,176285,2019-05-24,...,27,,GUBERN,PAPEL GUBERNAMENTAL,7.36,MXP,6.529709e+08,0.000000e+00,6.529709e+08,2019-05-24
1904351,1064295,True,Moral Nacional no Gravable,PRINCIPAL PENSIONES CAPITA,PRINCIPAL PENSIONES S.A. DE C.V. CAPITAL T,PPE990528GD0,1064295,1064295,175108,2019-05-23,...,27,,GUBERN,PAPEL GUBERNAMENTAL,8.26,MXP,5.520562e+04,0.000000e+00,5.520562e+04,2019-05-23
1904352,1063782,True,Moral Nacional no Gravable,AZT8589,SIEFORE AZTECA BASICA 85-89 S.A. DE C.V.,SAB080110UG5,1063782,1063782,894775,2018-07-05,...,6,,GUBERN,PAPEL GUBERNAMENTAL,7.84,MXP,2.522300e+08,0.000000e+00,2.522300e+08,2018-07-05


In [88]:
df.columns

Index(['NUMERO_CLIENTE', 'ESPROVEED', 'TIPO_PERSONA', 'NOMBRE', 'NOMLARGO',
       'RFC', 'NLINEA', 'ICONTRATO', 'IORDEN', 'FOPER', 'AÑO_OPE', 'MES_OPE',
       'DIA_OPE', 'FLIQ', 'AÑO_FLIQ', 'MES_FLIQ', 'DIA_FLIQ', 'CPZO_DE',
       'CPZO_A', 'PLAZOREF', 'FVENCE', 'AÑO_FVENCE', 'MES_FVENCE',
       'DIA_FVENCE', 'IINSTR', 'ITINSTR', 'NTINSTR', 'TASA', 'MONEDA', 'MONTO',
       'MONTO_ASIGNADO', 'MONTO_REAL', 'FECHA_REGISTRO'],
      dtype='object')

In [80]:
df

,NUMERO_DE_GARANTIA,NUMERO_CLIENTE,CLIENTE,DESC_CORTA,SUBPRODUCTO,FECHA_REGISTRO_GARANTIA,MONTO_CREDITO,PORCENTAJE_GARANTIZADO,MONTO_GARANTIZADO,TASA_INTERES,PORCENTAJE_COMISION,GANANCIA,AIO,MES,DIA
0,17349041,492,NACIONAL FINANCIERA,NAFIN,PRIMER PISO,2002-12-31 00:00:00,300000.0,100.0,300000.0,10.00,0.00,0,2002,12,31
1,17927149,492,NACIONAL FINANCIERA,NAFIN,PRIMER PISO,2002-12-31 00:00:00,350000.0,100.0,350000.0,10.00,0.00,0,2002,12,31
2,17756164,492,NACIONAL FINANCIERA,NAFIN,PRIMER PISO,2002-12-31 00:00:00,300000.0,100.0,300000.0,10.00,0.00,0,2002,12,31
3,18008738,492,NACIONAL FINANCIERA,NAFIN,PRIMER PISO,2002-12-31 00:00:00,200000.0,100.0,200000.0,10.00,0.00,0,2002,12,31
4,18664323,492,NACIONAL FINANCIERA,NAFIN,PRIMER PISO,2002-12-31 00:00:00,150000.0,100.0,150000.0,10.00,0.00,0,2002,12,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028094,89900773,285,BANCO MERCANTIL DEL NORTE,BANORTE,EMPRESARIAL,2024-12-31 00:00:00,1300000.0,50.0,650000.0,21.50,5.45,70850,2024,12,31
1028095,89900931,285,BANCO MERCANTIL DEL NORTE,BANORTE,EMPRESARIAL,2024-12-31 00:00:00,4000000.0,50.0,2000000.0,16.48,3.70,148000,2024,12,31
1028096,89901284,285,BANCO MERCANTIL DEL NORTE,BANORTE,EMPRESARIAL,2024-12-31 00:00:00,5500000.0,50.0,250000.0,7.00,3.70,203500,2024,12,31
1028097,89901463,285,BANCO MERCANTIL DEL NORTE,BANORTE,EMPRESARIAL,2024-12-31 00:00:00,1211000.0,50.0,605500.0,8.50,4.35,52679,2024,12,31


In [ ]:
col_sirac = [
    "NUMERO_PRESTAMO",
    "FECHA_APERTURA",
    "FECHA_VENCIMIENTO",
    "PRODUCTO",
    "DESC_LINEA_FINANCIERA",
    "MONTO_APROBADO",
    "MONTO_INICIAL",
    "TASA_TOTAL",
    "PLAZO",
    "AMORTIZACION",
    "NUMERO_CUOTAS",
    "VALOR_CUOTA",
    "CANTIDAD_CUOTAS_PAGADAS"
]

col_siag = [
    'NUMERO_DE_GARANTIA', 
    'SUBPRODUCTO', 
    'FECHA_REGISTRO_GARANTIA', 
    'MONTO_CREDITO',
    'PORCENTAJE_GARANTIZADO', 
    'MONTO_GARANTIZADO', 
    'TASA_INTERES',
    'PORCENTAJE_COMISION'
]

In [37]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat")

/tmp/ipykernel_24838/550213403.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/productos/TAS/data/26_02_2025/productos_clientes_TAS_T.dat")


In [38]:
df.head(5)

,NUMERO_CLIENTE,ESPROVEED,TIPO_PERSONA,NOMBRE,NOMLARGO,RFC,NLINEA,ICONTRATO,IORDEN,FOPER,...,DIA_FVENCE,IINSTR,ITINSTR,NTINSTR,TASA,MONEDA,MONTO,MONTO_ASIGNADO,MONTO_REAL,FECHA_REGISTRO
0,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,1040153,1040153,430215,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,3.302570e+09,3.302570e+09,2020-11-12
1,51000,False,Cuentas Propias,GARANTIAS CREDITO,GARANTIAS CREDITO,NFI3406305T0,51000,51000,430214,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.297121e+09,3.297121e+09,3.297121e+09,2020-11-12
2,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,1040153,1040153,430100,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,0.000000e+00,3.308033e+09,2020-11-12
3,51000,False,Cuentas Propias,GARANTIAS CREDITO,GARANTIAS CREDITO,NFI3406305T0,51000,51000,430063,2020-11-12,...,25,DEPBANX2,OTROS,Otros Instrumentos,4.25,MXP,3.302570e+09,0.000000e+00,3.308033e+09,2020-11-12
4,1040153,True,Moral Nacional no Gravable,BANXICO,BANCO DE MEXICO,BNM840515VB2,1040153,1040153,329069,2020-04-01,...,25,DEPBANX2,OTROS,Otros Instrumentos,6.50,MXP,5.217592e+09,5.217592e+09,5.217592e+09,2020-04-01


In [8]:
df_filtrado = df_clientes[df_clientes['SISTEMA_ORIGEN'].str.contains(',')]

In [11]:
df_filtrado

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION,SOXID
248,246,INF NO DISP,"AFIX SERVICIOS FINANCIEROS, S.A.P.I. DE C.V. S...","28563824, 1065491",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,ASF0901169V3,SIN INFO,SIN INFO,HEREDIA HORNER HUGO,"PM, Moral Nacional no Gravable",INF NO DISP,"SIRAC, TAS",11/04/25,"[S:2, I:8, R:5, A:6, C:3, ,:8, :2, T:4, A:,, ..."
533,531,INF NO DISP,"ALSOL CONTIGO, S.A. DE C.V., SOFOM, E.N.R.","28627541, 1065418, 1065564",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,ACO090216551,SIN INFO,SIN INFO,GONZALEZ GARCIA DARIO -- GONZALEZ GARCIA JOSE ...,"PM, Moral Nacional Gravable",INF NO DISP,"SIRAC, TAS",11/04/25,"[S:2, I:8, R:6, A:2, C:7, ,:5, :4, T:1, A:,, ..."
811,809,INF NO DISP,APOYO ECONOMICO FAMILIAR SA DE CV SOFOM ENR,"22336018, 1064485",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,AEF050221TY2,SIN INFO,SIN INFO,MESSMACHER HENRIQUEZ EDUARDO BERNHART -- OROCI...,"PM, Moral Nacional no Gravable",INF NO DISP,"SIRAC, TAS",11/04/25,"[S:2, I:2, R:3, A:3, C:6, ,:0, :1, T:8, A:,, ..."
1146,1140,INF NO DISP,"BANCA AFIRME, S.A.","306, 1041222",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,BAF950102JP5,SIN INFO,SIN INFO,ARZATE ORTIZ RAUL ROSENDO -- ASMITIA ANCONA JO...,"PM, Moral Nacional no Gravable",INF NO DISP,"TAS, SIPE",11/04/25,"[T:3, A:0, S:6, ,:,, : , S:1, I:0, P:4, E:1]"
1150,1144,INF NO DISP,"BANCA CREMI,S.A.","BCDI, 77",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,"MECA, SIPE",11/04/25,"[M:B, E:C, C:D, A:I, ,:,, : , S:7, I:7]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11835,11727,INF NO DISP,"SOFIPA CORPORATION, S.A.P.I. DE C.V., SOFOM E....","28917282, 1065803",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,FIN080623RZ8,SIN INFO,SIN INFO,GARCIA VERA GABRIEL -- HERNANDEZ DIAZ FRANCISCO,"PM, Moral Nacional no Gravable",INF NO DISP,"SIRAC, TAS",11/04/25,"[S:2, I:8, R:9, A:1, C:7, ,:2, :8, T:2, A:,, ..."
12378,12885,INF NO DISP,TRATON FINANCIAL SERVICES MEXICO S.A. DE C.V. ...,"4581019, 90066",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,MEXICO,...,NO SE TIENE REGISTRADA LA INFORMACION,NFI980108PX5,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,"PM, UNIONES DE CREDITO",INF NO DISP,"SIAG, SIRAC",11/04/25,"[S:4, I:5, A:8, G:1, ,:0, :1, S:9, I:,, R: , ..."
12423,12311,INF NO DISP,UBS AG,"UBS, 5",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,INF NO DISP,"SIPE, SIMS",11/04/25,"[S:U, I:B, P:S, E:,, ,: , :5]"
12510,12392,UNIDAD DE MERCADO DE CAPITALES,"VALORES MEXICANOS CASA DE BOLSA, S.A. DE C.V.","VALMEX, 1057519, 315",VIGENTE,INF NO DISP,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,INF NO DISP,INF NO DISP,...,,"CBC79081421A, VMC930101JS6, XXX123456XXX",SIN INFO,SIN INFO,ALVAREZ TOLEDO CLAUDIA CECILIA -- PAYAN LOPEZ ...,"PM, Moral Nacional no Gravable",INF NO DISP,"MECA, TAS",11/04/25,"[M:V, E:A, C:L, A:M, ,:E, :X, T:,, A: , S:1]"


In [14]:
df_filtrado["SOXID"] = df_filtrado.apply(
    lambda row: {f"{so}:{id}" for so, id in zip(row["SISTEMA_ORIGEN"].split(","), row["NUMERO_CLIENTE"].split(","))},
    axis=1
)


/tmp/ipykernel_24838/3449657443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["SOXID"] = df_filtrado.apply(


In [15]:
df_filtrado["SOXID"] 

248      {SIRAC:28563824,  TAS: 1065491}
533      { TAS: 1065418, SIRAC:28627541}
811      { TAS: 1064485, SIRAC:22336018}
1146           { SIPE: 1041222, TAS:306}
1150              {MECA:BCDI,  SIPE: 77}
                      ...               
11835    { TAS: 1065803, SIRAC:28917282}
12378      { SIRAC: 90066, SIAG:4581019}
12423               {SIPE:UBS,  SIMS: 5}
12510       {MECA:VALMEX,  TAS: 1057519}
12547         {MECA:VECT,  TAS: 1057520}
Name: SOXID, Length: 91, dtype: object